In [1]:
import csv
import json
import requests


In [2]:
def analyze_sentiment(articles):
    api_endpoint = "https://language.googleapis.com/v1/documents:analyzeSentiment"
    api_key = ""  # Replace with your actual API key
    
    sentiment_results = []  # List to store sentiment analysis results
    
    for idx, article in enumerate(articles, start=1):
        text_content = article["Text Content"]
        article_title = article["Article Title"]
        
        # Create the request payload for each article
        data = {
            "document": {
                "type": "PLAIN_TEXT",
                "content": text_content
            },
            "encodingType": "UTF8"
        }
        
        params = {
            "key": api_key
        }
        
        headers = {
            "Content-Type": "application/json"
        }
        
        # Send the HTTP POST request for each article
        response = requests.post(api_endpoint, params=params, headers=headers, data=json.dumps(data))
        
        if response.status_code == 200:
            sentiment_data = response.json()
            # Get overall sentiment of the input article
            document_sentiment = sentiment_data["documentSentiment"]
            document_result = {
                "Article Number": idx,
                "Article Title": article_title,
                "Sentiment Type": "Document",
                "Sentiment Score": document_sentiment["score"],
                "Sentiment Magnitude": document_sentiment["magnitude"]
            }
            sentiment_results.append(document_result)
        else:
            print(f"Error for Article {idx}: {response.status_code} - {response.text}")
            continue
    
    # Save the sentiment results to a CSV file
    csv_file = "sentiment_results.csv"
    csv_headers = ["Article Number", "Article Title", "Sentiment Type", "Sentiment Score", "Sentiment Magnitude"]
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=csv_headers)
        writer.writeheader()
        writer.writerows(sentiment_results)
    
    print(f"Sentiment results saved to {csv_file}")

if __name__ == "__main__":
    # Read articles and their text content from a CSV file
    articles = []
    with open('articles.csv', mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            articles.append({"Article Title": row['Article Title'], "Text Content": row['Text Content']})
    
    analyze_sentiment(articles)

Sentiment results saved to sentiment_results.csv
